In [ ]:
batches = #
keep_prob = #
length = #
nodes = #
epoch = #
init_mode =  #
reg = #

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# 데이터 불러오기
train = pd.read_csv('./data/train.csv', index_col=0)
test = pd.read_csv('./data/test.csv', index_col=0)
sample_submission = pd.read_csv('./data/sample_submission.csv', index_col=0)

# Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# 모델에 적용할 데이터 셋 준비 
x = train.drop(columns=['type', 'type_num'], axis=1)
y = train['type_num']

x = x.drop(columns=['fiberID'], axis=1)
test_x = test.drop(columns=['fiberID'],axis=1)

x_name=x.columns
col_name=x_name

In [ ]:
x=np.array(x)
y=np.array(y)
test_x=np.array(test_x)

# 이상치 처리 방법

In [ ]:
# plt.figure(figsize=(25,25))
# x1_da = x.copy()

# x_max = 31
# x_min = 12

# for i in range(20):
#     idx=(x1_da[:,i]> x_max) | (x1_da[:,i] < x_min)
    
#     val1=len(x1_da[idx,i])
#     val2=np.round(x1_da[~idx,i].mean(),4)
#     val3=np.round(x1_da[~idx,i].std(),4)
#     val4=np.round(np.median((x1_da[~idx,i])),4)
    
#     if i ==0: 
#         print("특정 조건을 선택한 경우\n")
#         print('{:>12}{:>8}{:>9}{:>9}{:>11}'.format('COLUMN','Count','  Mean', 'Std', 'Median'))
#     print('{:>12}{:>8}{:>9}{:>9}{:>11}'.format(col_name[i],val1,val2,val3,val4))


#     #######################
#     #   이상치 제거
#     #
#     # x1_da[idx,i]=val4
#     #
#     #######################
    
# #     plt.subplot(4,5,i+1)
# #     plt.hist(x1_da[:,i],bins=50)
# #     plt.gca().set_title(col_name[i])
   

------------------------

In [ ]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.9, shuffle=True ,random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# scaler1=RobustScaler()

# scaler1.fit(x_train)
# x_train=scaler1.transform(x_train)
# x_test=scaler1.transform(x_test)

#test_x =scaler1.transform(test_x)

scaler2=StandardScaler()

scaler2.fit(x_train)
x_train=scaler2.transform(x_train)
x_test=scaler2.transform(x_test)

test_x =scaler2.transform(test_x)

In [ ]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
x=x.astype('float32')
test_x=test_x.astype('float32')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation

In [ ]:
inputs=Input(shape=(20,), name='input')

x=Dense(64,activation='elu',name='hidden0')(inputs)
x=Dropout(0.2)(x)

for i in range(length):
    if i%2 ==0:
        x=Dense(nodes*2, activation='elu',name='hidden'+str(i+1), \
                   kernel_initializer=init_mode,\
                   kernel_regularizer=regularizers.l2(reg))(x)       
        print(i)
    elif i%2 == 1:
        x=Dense(nodes, activation='elu',name='hidden'+str(i+1), \
                   kernel_initializer=init_mode,\
                   kernel_regularizer=regularizers.l2(reg))(x)       
        print(i)
    
    if i == 5-2:
        x=Dropout(keep_prob)(x)
        
prediction=Dense(19,activation='softmax',name='output')(x)
model = Model(inputs=inputs, outputs=prediction)

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hist = model.fit(x_train, y_train, batch_size= batches, epochs=epoch, verbose=2, validation_split=1./9.)

loss, acc = model.evaluate(x_test,y_test,batch_size= 256)
print('Loss:',loss,'Accuracy:',acc)

In [ ]:
y_pred = model.predict_proba(test_x)

In [ ]:
import matplotlib.pyplot as plt

print(hist.history.keys())

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper right')

plt.show()

In [ ]:
import os.path
i=0
while 1:
    if os.path.isfile('./data/best_params'+str(i)+'.txt') == True:
        i+=1
    else:
        break
     
# 그림 생성
plt.savefig('./data/res'+str(i)+'.png')

# 제출 파일 생성
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('./data/submission_data_'+str(i)+'.csv', index=True)